# **`Redis Installation` & Setup for `Development` and `Production`:**

This guide describes how to install and run Redis using Docker Compose in two environments: **development** (with a friendly web UI) and **production** (CLI‑only, hardened for security). It assumes that you already have Docker and Docker Compose installed on your system.

## **`Overview`:**

`Redis` is designed to run in trusted environments and should not be exposed directly to the internet. The official documentation states that Redis should be accessed only by trusted clients; exposing the Redis TCP port to untrusted networks can allow remote attackers to delete data or run dangerous commands. Authentication via `requirepass` or Access Control Lists (ACLs) adds a layer of protection by forcing clients to authenticate before sending commands. For durability, Redis provides an **`Append Only File (AOF)`** mechanism: enabling `appendonly yes` records every write operation so the database can rebuild its state on restart.



### **Prerequisites:**
* **Docker & Docker Compose:**
    * Verify installation:
        ```bash
            docker --version
            docker compose version
        ```

* **Project directory:**
    * Create a directory for your Redis project and change into it:
        ```bash
            mkdir redis-setup && cd redis-setup
        ```

* **Environment file:**
    * Create a `.env` file to store secrets (such as the Redis password) outside of version control:
        ```env
            # .env
            REDIS_PASSWORD=change_me_to_a_strong_password
        ```

* **Directory structure:**
    * Your project should look like this:
        ```graphql
            redis-setup/
                ├── docker-compose.yml   # Compose file (different for dev and prod)
                ├── .env                 # secrets (never commit this to version control)
                └── redis-data/          # persistent data volume (created automatically)
        ```

The `redis-data` folder is mounted into the container to ensure durability of the AOF files and snapshots across restarts. Redis writes every dataset‑changing command to the AOF when `appendonly yes` is enabled.

## **`Development Environment`:**

In development you may want an easy way to explore data structures or monitor commands. **`Redis Stack`** includes Redis modules (JSON, Search, Bloom, TimeSeries, etc.) and the **`RedisInsight`** web UI. The following Compose file publishes both the Redis port and the web UI port.

### **`docker-compose.yml` (development):**
```yaml
    version: "3.8"

    services:
    redis-stack:
        image: redis/redis-stack:latest
        container_name: redis-stack
        ports:
        - "6379:6379"   # Redis TCP port (development only)
        - "8001:8001"   # RedisInsight web UI
        volumes:
        - ./redis-data:/data
        environment:
        REDIS_ARGS: "--appendonly yes --requirepass ${REDIS_PASSWORD}"
        restart: unless-stopped
```

### **Why this works:**
* **Web UI for learning -** the RedisInsight UI runs on port 8001 and allows you to inspect keys or run queries. This should be enabled only in development.
* **AOF persistence –** --`appendonly yes` ensures every write is appended to a log so the state can be replayed at startup.
* **Password protection –** `requirepass` rejects unauthenticated clients; clients must authenticate using `AUTH` to run commands.
* **Environment variables –** the password is supplied through `${REDIS_PASSWORD}` from the `.env` file, so secrets are not hard‑coded.



### **Running and verifying (development):**
* **Start the stack:**
    ```bash
        docker compose --env-file .env up -d
    ```


* **Access the RedisInsight UI:** visit `http://localhost:8001` in your browser and connect to the Redis server using:
    | Field    | Value               |
    | -------- | ------------------- |
    | Host     | `localhost`         |
    | Port     | `6379`              |
    | Password | `${REDIS_PASSWORD}` |


* **Test with `redis-cli`:** you can also connect using the command line. The Redis CLI documentation shows that to connect remotely you provide the hostname, port and password.
    ```bash
        redis-cli -h localhost -p 6379 -a "${REDIS_PASSWORD}"
        > PING
        PONG
    ```

Alternatively, set the password once via `REDISCLI_AUTH`:
    ```bash
        export REDISCLI_AUTH=${REDIS_PASSWORD}
        redis-cli -h localhost -p 6379 PING
        PONG
    ```


### **Stopping and cleanup (development):**
* To stop and remove containers while keeping the data volume:
    ```bash
        docker compose down
    ```

* To remove data as well (development only):
    ```bash
        docker compose down -v
    ```

## **`Production Environment`:**